In [1]:
%pip install datasets

import copy
import ast
import pandas as pd
import numpy as np
from functools import partial
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import KFold
from typing import List, Dict, Union, Any
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [2]:
class CT4A_DataLoader:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def get_dataset(self, dataset_path):
        data = pd.read_csv(dataset_path, sep='\t')
        data['labels'] = data['labels'].apply(lambda x: ast.literal_eval(x))

        def tokenize(examples):
            return self.tokenizer(examples["text"], max_length=128, truncation=True, padding='max_length')

        dataset = Dataset.from_pandas(data[['text', 'labels']])
        dataset = dataset.map(tokenize, batched=True, batch_size=32, remove_columns=['text'])

        return dataset, data

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_metrics(data):
    metrics = {}
    for i, _cat in enumerate(['cat1', 'cat2', 'cat3']):
        preds = data[f'{_cat}_pred'].apply(lambda x: int(x) == 1)
        labels = data['labels'].apply(lambda x: int(float(x[i])) == 1)

        acc = accuracy_score(labels, preds)
        prec = precision_score(labels, preds)
        rec = recall_score(labels, preds)
        f1 = f1_score(labels, preds)
        metrics.update({f'{_cat}_avg_acc': acc,
                        f'{_cat}_avg_prec': prec,
                        f'{_cat}_avg_rec': rec,
                        f'{_cat}_avg_f1': f1
                        })

    preds = data[[c for c in data.columns if '_pred' in c]].apply(lambda x: list([float(p) for p in x]), axis=1).tolist()
    labels = data['labels'].tolist()
    metrics["macro_f1"] = f1_score(labels, preds, average="macro")

    return metrics

In [10]:
def annotate_test_dataframe(data, pred_output):
    data['cat1_logits'] = pred_output.predictions[:, 0]
    data['cat2_logits'] = pred_output.predictions[:, 1]
    data['cat3_logits'] = pred_output.predictions[:, 2]

    predictions = (pred_output.predictions > 0) * 1
    data['cat1_pred'] = predictions[:, 0]
    data['cat2_pred'] = predictions[:, 1]
    data['cat3_pred'] = predictions[:, 2]

    data['cat1_score'] = sigmoid(pred_output.predictions[:, 0])
    data['cat2_score'] = sigmoid(pred_output.predictions[:, 1])
    data['cat3_score'] = sigmoid(pred_output.predictions[:, 2])

    return data

In [5]:
path_or_id = "microsoft/deberta-v3-base"
model_id_or_path = path_or_id
tokenizer_id_or_path = path_or_id

epochs = 10
learning_rate = 2e-5
tokenizer_max_len = 128

dataloader_config = {'per_device_train_batch_size': 16,
                     'per_device_eval_batch_size': 256}

In [6]:
print('load model and tokenizer')
tokenizer_config = {'pretrained_model_name_or_path': tokenizer_id_or_path,
                    'max_len': tokenizer_max_len}

tokenizer = AutoTokenizer.from_pretrained(**tokenizer_config)

model_config = {'pretrained_model_name_or_path': model_id_or_path,
                'num_labels': 3,
                'problem_type': "multi_label_classification"}

model = AutoModelForSequenceClassification.from_pretrained(**model_config)

load model and tokenizer


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(f'load data')
dl = CT4A_DataLoader(tokenizer)
train_ds, train_df = dl.get_dataset("ct_train.tsv")
dev_ds, dev_df = dl.get_dataset("ct_dev.tsv")

load data


Map:   0%|          | 0/1229 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir=f"results",
    num_train_epochs=epochs,
    **dataloader_config,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=learning_rate,
    logging_strategy='no',
    save_strategy='no',
    eval_strategy="no",
    no_cuda=False,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=770, training_loss=0.19247957700258725, metrics={'train_runtime': 356.2969, 'train_samples_per_second': 34.494, 'train_steps_per_second': 2.161, 'total_flos': 808430473843200.0, 'train_loss': 0.19247957700258725, 'epoch': 10.0})

In [11]:
dev_pred_output = trainer.predict(dev_ds)
dev_df = annotate_test_dataframe(dev_df, dev_pred_output)
print(compute_metrics(dev_df))

{'cat1_avg_acc': 0.927007299270073, 'cat1_avg_prec': 0.7666666666666667, 'cat1_avg_rec': 0.8846153846153846, 'cat1_avg_f1': 0.8214285714285714, 'cat2_avg_acc': 0.9197080291970803, 'cat2_avg_prec': 0.7777777777777778, 'cat2_avg_rec': 0.8076923076923077, 'cat2_avg_f1': 0.7924528301886793, 'cat3_avg_acc': 0.948905109489051, 'cat3_avg_prec': 0.8857142857142857, 'cat3_avg_rec': 0.9117647058823529, 'cat3_avg_f1': 0.8985507246376812, 'macro_f1': 0.8374773754183106}


In [26]:
submission_df = dev_df[["index", "cat1_pred", "cat2_pred", "cat3_pred"]]
submission_df.to_csv("predictions.csv")